In [24]:
import pandas as pd
import matplotlib.pyplot as plt
from cobra.io import read_sbml_model
model = read_sbml_model('iML1515.xml.gz')
model

Name,iML1515
Memory address,0x07f97cce29550
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [25]:
model_sucd= model.copy()
model_sucd

Name,iML1515
Memory address,0x07f97cce294c0
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [26]:
from cobra import Reaction, Metabolite
sucd = Reaction('SUCD') #CoA-dependent succinate semialdehyde dehydrogenase
succinyl_semialdehyde = Metabolite(id='succinyl_semialdehyde_c', compartment='c')

sucd.add_metabolites({model_sucd.metabolites.h_c: 1,
                              model_sucd.metabolites.nadh_c: -1,
                              model_sucd.metabolites.succoa_c: -1,
                              succinyl_semialdehyde: 1, #new one
                              model_sucd.metabolites.coa_c: 1,
                              model_sucd.metabolites.nad_c: 1
                             })
print(sucd.build_reaction_string())
model_sucd.add_reactions([sucd])
model_sucd.reactions.SUCD

nadh_c + succoa_c --> coa_c + h_c + nad_c + succinyl_semialdehyde_c


Reaction identifier,SUCD
Name,
Memory address,0x07f97cd818b80
Stoichiometry,nadh_c + succoa_c --> coa_c + h_c + nad_c + succinyl_semialdehyde_c Nicotinamide adenine dinucleotide - reduced + Succinyl-CoA --> Coenzyme A + H+ + Nicotinamide adenine dinucleotide +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [27]:
from cobra import Reaction, Metabolite
fhbd = Reaction('FHBD') #4-hydroxybutyrate dehydrogenase
four_hydroxybutyrate = Metabolite(id='four_hydroxybutyrate_c', compartment='c')

fhbd.add_metabolites({model_sucd.metabolites.h_c: 1,
                              model_sucd.metabolites.nadh_c: -1,
                              succinyl_semialdehyde: -1, 
                              four_hydroxybutyrate: 1, #new one
                              model_sucd.metabolites.nad_c: 1
                             })
print(fhbd.build_reaction_string())
model_sucd.add_reactions([fhbd])
model_sucd.reactions.FHBD

nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c


Reaction identifier,FHBD
Name,
Memory address,0x07f97cd8189a0
Stoichiometry,nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c Nicotinamide adenine dinucleotide - reduced + --> + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [28]:
from cobra import Reaction, Metabolite
hbd_coa_t = Reaction('HBCT') #4-hydroxybutyryl-CoA transferase
four_hbcoa = Metabolite(id='four_hbcoa_c', compartment='c')
hbd_coa_t.add_metabolites({model_sucd.metabolites.accoa_c: -1,
                              model_sucd.metabolites.ac_c: 1,
                              four_hbcoa: 1, #new one
                              four_hydroxybutyrate: -1
                             })
print(hbd_coa_t.build_reaction_string())
model_sucd.add_reactions([hbd_coa_t])
model_sucd.reactions.HBCT

accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c


Reaction identifier,HBCT
Name,
Memory address,0x07f97cd810af0
Stoichiometry,accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c Acetyl-CoA + --> Acetate +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [29]:
from cobra import Reaction, Metabolite
hbd_coa_r = Reaction('HBCR') #4-hydroxybutyryl-CoA reductase
four_hbaldehyde = Metabolite(id='four_hbaldehyde_c', compartment='c')
hbd_coa_r.add_metabolites({model_sucd.metabolites.h_c: 1,
                              model_sucd.metabolites.nadh_c: -1,
                              four_hbaldehyde: 1, #new one
                              four_hbcoa: -1,
                              model_sucd.metabolites.nad_c: 1,
                              model_sucd.metabolites.coa_c: 1
                             })
print(hbd_coa_r.build_reaction_string())
model_sucd.add_reactions([hbd_coa_r])
model_sucd.reactions.HBCR

four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c


Reaction identifier,HBCR
Name,
Memory address,0x07f97cd810040
Stoichiometry,four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c + Nicotinamide adenine dinucleotide - reduced --> Coenzyme A + + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [30]:
from cobra import Reaction, Metabolite
adh= Reaction('ADH') #alcohol dehydrogenase
bdo = Metabolite(id='bdo_c', compartment='c')
adh.add_metabolites({model_sucd.metabolites.h_c: 1,
                              model_sucd.metabolites.nadh_c: -1,
                              four_hbaldehyde: -1,
                              bdo: 1, #new one
                              model_sucd.metabolites.nad_c: 1,
                             })
print(adh.build_reaction_string())
model_sucd.add_reactions([adh])
model_sucd.reactions.ADH

four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c


Reaction identifier,ADH
Name,
Memory address,0x07f97cd8109a0
Stoichiometry,four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c + Nicotinamide adenine dinucleotide - reduced --> + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [31]:
bdo_exchange = Reaction('EX_bdo')
bdo_exchange.add_metabolites({model_sucd.metabolites.bdo_c: -1})
model_sucd.add_reaction(bdo_exchange)
model_sucd.add_boundary(model_sucd.metabolites.bdo_c, type='demand')

Reaction identifier,DM_bdo_c
Name,demand
Memory address,0x07f97cd8185e0
Stoichiometry,bdo_c --> -->
GPR,
Lower bound,0
Upper bound,1000.0


In [35]:
with model:
    model_sucd.objective=model_sucd.reactions.ADH
    optimized = float(model_sucd.optimize().objective_value)
    print(optimized)

9.605894736842117


In [37]:
uptake = model_sucd.reactions.EX_glc__D_e.flux #uptake <--
print(uptake)

-10.0


In [41]:
#input text
Models_yield = dict()
Models_yield['one']= optimized
 
#open file
file = open("SucD Yield calc.txt", "w")
 
#convert variable to string
str = repr(Models_yield)
file.write("Models_yield = " + str + "\n")
 
#close file
file.close()
 
f = open('SucD Yield calc.txt', 'r')
if f.mode=='r':
    contents= f.read()

In [43]:
print("Yield for model SucD =",(-1)*((Models_yield['one'])/uptake)) # Yield for Model_SucD (Succinate) p-mmol/gl-mmol
#it is multiplied by -1 because the reaction is an uptake which means thats goes for the right to the left

Yield for model SucD = 0.9605894736842118


In [13]:
model_akg=model.copy()
model_akg

Name,iML1515
Memory address,0x07f97c80e3b20
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [14]:
from cobra import Reaction, Metabolite
oxodc = Reaction('OXODC') #2-oxoglutarate decarboxylase
succinyl_semialdehyde = Metabolite(id='succinyl_semialdehyde_c', compartment='c')

oxodc.add_metabolites({model_akg.metabolites.akg_c: -1,
                              succinyl_semialdehyde: 1, #new one
                              model_akg.metabolites.co2_c: 1
                             })
print(oxodc.build_reaction_string())
model_akg.add_reactions([oxodc])
model_akg.reactions.OXODC

akg_c --> co2_c + succinyl_semialdehyde_c


Reaction identifier,OXODC
Name,
Memory address,0x07f97cce29d90
Stoichiometry,akg_c --> co2_c + succinyl_semialdehyde_c 2-Oxoglutarate --> CO2 CO2 +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [15]:
from cobra import Reaction, Metabolite
fhbd = Reaction('FHBD') #4-hydroxybutyrate dehydrogenase
four_hydroxybutyrate = Metabolite(id='four_hydroxybutyrate_c', compartment='c')

fhbd.add_metabolites({model_akg.metabolites.h_c: 1,
                              model_akg.metabolites.nadh_c: -1,
                              succinyl_semialdehyde: -1, 
                              four_hydroxybutyrate: 1, #new one
                              model_akg.metabolites.nad_c: 1
                             })
print(fhbd.build_reaction_string())
model_akg.add_reactions([fhbd])
model_akg.reactions.FHBD

nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c


Reaction identifier,FHBD
Name,
Memory address,0x07f97cce2f5b0
Stoichiometry,nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c Nicotinamide adenine dinucleotide - reduced + --> + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [16]:
from cobra import Reaction, Metabolite
hbd_coa_t = Reaction('HBCT') #4-hydroxybutyryl-CoA transferase
four_hbcoa = Metabolite(id='four_hbcoa_c', compartment='c')
hbd_coa_t.add_metabolites({model_akg.metabolites.accoa_c: -1,
                              model_akg.metabolites.ac_c: 1,
                              four_hbcoa: 1, #new one
                              four_hydroxybutyrate: -1
                             })
print(hbd_coa_t.build_reaction_string())
model_akg.add_reactions([hbd_coa_t])
model_akg.reactions.HBCT

accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c


Reaction identifier,HBCT
Name,
Memory address,0x07f97cce2fca0
Stoichiometry,accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c Acetyl-CoA + --> Acetate +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [17]:
from cobra import Reaction, Metabolite
hbd_coa_r = Reaction('HBCR') #4-hydroxybutyryl-CoA reductase
four_hbaldehyde = Metabolite(id='four_hbaldehyde_c', compartment='c')
hbd_coa_r.add_metabolites({model_akg.metabolites.h_c: 1,
                              model_akg.metabolites.nadh_c: -1,
                              four_hbaldehyde: 1, #new one
                              four_hbcoa: -1,
                              model_akg.metabolites.nad_c: 1,
                              model_akg.metabolites.coa_c: 1
                             })
print(hbd_coa_r.build_reaction_string())
model_akg.add_reactions([hbd_coa_r])
model_akg.reactions.HBCR

four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c


Reaction identifier,HBCR
Name,
Memory address,0x07f97cce2f970
Stoichiometry,four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c + Nicotinamide adenine dinucleotide - reduced --> Coenzyme A + + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [18]:
from cobra import Reaction, Metabolite
adh= Reaction('ADH') #alcohol dehydrogenase
bdo = Metabolite(id='bdo_c', compartment='c')
adh.add_metabolites({model_akg.metabolites.h_c: 1,
                              model_akg.metabolites.nadh_c: -1,
                              four_hbaldehyde: -1,
                              bdo: 1, #new one
                              model_akg.metabolites.nad_c: 1,
                             })
print(adh.build_reaction_string())
model_akg.add_reactions([adh])
model_akg.reactions.ADH

four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c


Reaction identifier,ADH
Name,
Memory address,0x07f97cce3f940
Stoichiometry,four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c + Nicotinamide adenine dinucleotide - reduced --> + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [19]:
bdo_exchange = Reaction('EX_bdo')
bdo_exchange.add_metabolites({model_akg.metabolites.bdo_c: -1})
model_akg.add_reaction(bdo_exchange)
model_akg.add_boundary(model_akg.metabolites.bdo_c, type='demand')

Reaction identifier,DM_bdo_c
Name,demand
Memory address,0x07f97cce3f850
Stoichiometry,bdo_c --> -->
GPR,
Lower bound,0
Upper bound,1000.0


In [49]:
with model:
    model_akg.objective=model_akg.reactions.ADH
    optimized = float(model_akg.optimize().objective_value)
    print(optimized)

9.708085106382972


In [50]:
uptake_akg = model_akg.reactions.EX_glc__D_e.flux #uptake <--
print(uptake_akg)

-10.0


In [51]:
#input text
Models_yield = dict()
Models_yield['one']= optimized
 
#open file
file = open("Akg Yield calc.txt", "w")
 
#convert variable to string
str = repr(Models_yield)
file.write("Models_yield = " + str + "\n")
 
#close file
file.close()
 
f = open('Akg Yield calc.txt', 'r')
if f.mode=='r':
    contents= f.read()

In [52]:
print("Yield for model Akg =",(-1)*((Models_yield['one'])/uptake)) # Yield for Model_akg (a-ketoglutarate) p-mmol/gl-mmol
#it is multiplied by -1 because the reaction is an uptake which means thats goes for the right to the left

Yield for model Akg = 0.9708085106382972
